In [1]:
import torch
from torch import nn
from torch import optim
from torch.nn import functional as F
import numpy as np

In [2]:
a = torch.randn(1500, 5)
e = torch.randn(1500)

In [3]:
b = e*0.1 + torch.sin(a[:, 0]*10 + 20*a[:, 1]**3) + \
    torch.cos((3*a[:, 2] + 5*a[:, 3]**2)) + \
    2/(torch.sqrt(1 + torch.relu(a[:, 4] + 0.05))) + \
    2 * a[:, 4] * a[:, 0]

In [4]:
def loss(y, b):
    return torch.square(y-b).mean()

# def loss(y, b):
#     use_loss = torch.square(y-b).mean()
#     return (use_loss * torch.clamp(torch.exp(use_loss), max=10)).mean()

In [5]:
b.mean(), b.std()

(tensor(1.8433), tensor(2.1302))

In [6]:
aa = a[1000:]
bb = b[1000:]
a = a[:1000]
b = b[:1000]

In [7]:
class Multiply(nn.Module):
    def __init__(self, factor):
        super(Multiply, self).__init__()
        self.factor = factor

    def forward(self, x):
        return x*self.factor

In [8]:
print (aa.shape, bb.shape, a.shape, b.shape)

torch.Size([500, 5]) torch.Size([500]) torch.Size([1000, 5]) torch.Size([1000])


In [9]:
device = 'cuda:0'
all_res = []
all_losses_alpha_more2 = {}
all_test_losses_alpha_more2 = {}
m = 5000
a = a.to(device)
b = b.to(device)
aa = aa.to(device)
bb = bb.to(device)
j = 0
for j in range(10):
    torch.manual_seed(j*77+2951)
    for alpha in [-2.0, -1, 0.05, 0, 0.01]:
        z2 = nn.Sequential(nn.Linear(5, m), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, 1))
        nn.init.constant_(z2[0].bias, 0)
        nn.init.normal_(z2[0].weight, mean=0, std=np.sqrt(1./m))
        nn.init.constant_(z2[1].bias, 0)
        nn.init.normal_(z2[1].weight, mean=0, std=np.sqrt(2./m))
        nn.init.normal_(z2[4].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[4].bias, 0)
        nn.init.normal_(z2[7].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[7].bias, 0)
        nn.init.normal_(z2[10].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[10].bias, 0)
        nn.init.normal_(z2[13].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[13].bias, 0)
        nn.init.normal_(z2[16].weight, mean=0, std=np.sqrt(1))
        nn.init.constant_(z2[16].bias, 0)
        z2 = z2.to(device)
        all_losses2 = []
        all_test_losses2 = []
        op2 = optim.SGD(lr=1e-4, params=z2.parameters())
        print ("=============", alpha, "===============")
        for i in range(5000):
            op2.zero_grad()
            yh = z2(a).view(-1)
            loss_use = loss(yh, b)
            loss_use.backward()
            op2.step()
            all_losses2.append(loss_use.item())
            yh = z2(aa).view(-1)
            loss_use = loss(yh, bb)
            all_test_losses2.append(loss_use.item())
            if i % 5000 == 0:
                print (i, all_losses2[-1], all_test_losses2[-1])
            elif i < 500 and i % 100 == 0:
                print (i, all_losses2[-1], all_test_losses2[-1])
            if i % 500000 == 0:
                torch.save([all_losses_alpha_more2, all_test_losses_alpha_more2], f'v8_alpha_experiment_run2_{alpha}_{j}.pt')
        all_losses_alpha_more2[alpha] = all_losses2
        all_test_losses_alpha_more2[alpha] = all_test_losses2
        torch.save([all_losses_alpha_more2, all_test_losses_alpha_more2], f'v8_alpha_experiment_run2_{j}.pt')
        all_res.append([all_losses_alpha_more2, all_test_losses_alpha_more2])

In [10]:
alphas = [-2, -1, 0, 0.01, 0.05]
trains = {}
tests = {}

for alpha in alphas:
    trains[alpha] = []
    tests[alpha] = []

for j in range(10):
    a1, a2 = torch.load(f'v8_alpha_experiment_run2_{j}.pt')
    for alpha in alphas:
        trains[alpha].append(a1[alpha][1000])
        tests[alpha].append(a2[alpha][30])


In [11]:
for alpha in alphas:
    print (alpha, np.mean(trains[alpha]), np.mean(tests[alpha]))

In [83]:
for alpha in alphas:
    print (alpha, np.std(trains[alpha]), np.std(tests[alpha]))

-2 0.0023977802265394794 0.06740795828737217
-1 0.0015850432074508635 0.06514303480364823
0 0.013449423925627958 0.17348444542169325
0.01 0.011958419904337856 0.17462207729392137
0.05 0.022404042458352952 0.21832848256192608


In [89]:
a1, a2 = torch.load(f'v8_alpha_experiment_run2_0.pt')

In [14]:
markers = {-2:'o', -1:'s', 0:'^', 0.01:'<', 0.05:'>', -10:'v'}
colors = {-1:'C1',-2:'C0',0:'C2', 0.05:'C4',0.01:'C3'}
all_losses_alpha_more2 = a1
all_test_losses_alpha_more2 = a2

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
# check_keys = sorted([j for j in all_losses_alpha_more2.keys()])
check_keys = [-2, -1, 0, 0.01, 0.05]
j = 0
# markers = ['o', 's', '^', '<', '>', 'v', 'x', 'd', '*']
for key in sorted(check_keys):
    ax.plot(range(2, 1000), all_losses_alpha_more2[key][2:1000], f'{markers[key]}-', linewidth=3, 
            markersize=8, color=colors[key], markevery=240)
    j += 1
# ax.set_xscale('log')
ax.set_yscale('log')
ax.legend([f'alpha = {j}' for j in check_keys], fontsize=24)
ax.grid()
ax.set_xlabel("Number of Epochs", fontsize=24)
ax.set_ylabel("Training Error", fontsize=24)
# ax.set_ylim(0.9, 1.25e3)
ax.set_xticklabels(['', "0", "200", "400", "600", "800", "1,000"], fontsize=24)
ax.set_yticks([0.01, 0.1, 1, 10, 100, 1000])
ax.set_ylim(0.009, 1500)
ax.set_yticklabels(['$10^{-2}$',  '$10^{-1}$', '$10^{0}$', '$10^{1}$', '$10^{2}$', '$10^{3}$',], fontsize=24)
ax.set_title("")
fig.tight_layout()
fig.savefig('simulated_plot_train_v7', dpi=600)

In [15]:
markers = {-2:'o', -1:'s', 0:'^', 0.01:'<', 0.05:'>', -10:'v'}
colors = {-1:'C1',-2:'C0',0:'C2', 0.05:'C4',0.01:'C3'}
all_losses_alpha_more2 = a1
all_test_losses_alpha_more2 = a2

import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(8, 8))
# check_keys = sorted([j for j in all_losses_alpha_more2.keys()])
check_keys = [-2, -1, 0, 0.01, 0.05]
j = 0
for key in check_keys:
    ax.plot(range(2, 102), all_test_losses_alpha_more2[key][2:102], f'{markers[key]}-', linewidth=3, 
            markersize=8, color=colors[key], markevery=24)
    j += 1
# ax.set_xscale('log')
ax.set_yscale('log')
ax.legend([f'alpha = {j}' for j in check_keys], fontsize=24)
ax.grid()
ax.set_xlabel("Number of Epochs", fontsize=24)
ax.set_ylabel("Testing Error", fontsize=24)
ax.set_ylim(0.9, 1.25e3)
ax.set_xticklabels(['', "0", "20", "40", "60", "80", '100'], fontsize=24)
ax.set_yticklabels(['',  '',  '$10^{0}$', '$10^{1}$', '$10^{2}$', '$10^{3}$'], fontsize=24)
ax.set_title("")
fig.tight_layout()
fig.savefig('simulated_plot_test_v7', dpi=600)

### Next, run the experiments with many choice of alpha and get theoretically predicted upper bounds of the convergence rates and numerical convergence rates.

In [16]:
device = 'cuda:0'
# all_res = []
all_losses_alpha_more2 = {}
all_test_losses_alpha_more2 = {}
m = 5000
a = a.to(device)
b = b.to(device)
aa = aa.to(device)
bb = bb.to(device)
j = 0
for j in range(10):
    torch.manual_seed(j*77+2951)
    for alpha in [-2.0, -1, 0.05, 0, 0.01, -3, -5, -7, -10, -0.1, -0.5, 0.1, 0.2, 0.3, 0.4, 0.5]:
        z2 = nn.Sequential(nn.Linear(5, m), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, m), 
                          nn.LeakyReLU(alpha), 
                          Multiply(1./np.sqrt(1+alpha**2)), 
                          nn.Linear(m, 1))
        nn.init.constant_(z2[0].bias, 0)
        nn.init.normal_(z2[0].weight, mean=0, std=np.sqrt(1./m))
        nn.init.constant_(z2[1].bias, 0)
        nn.init.normal_(z2[1].weight, mean=0, std=np.sqrt(2./m))
        nn.init.normal_(z2[4].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[4].bias, 0)
        nn.init.normal_(z2[7].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[7].bias, 0)
        nn.init.normal_(z2[10].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[10].bias, 0)
        nn.init.normal_(z2[13].weight, mean=0, std=np.sqrt(2./m))
        nn.init.constant_(z2[13].bias, 0)
        nn.init.normal_(z2[16].weight, mean=0, std=np.sqrt(1))
        nn.init.constant_(z2[16].bias, 0)
        z2 = z2.to(device)
        all_losses2 = []
        all_test_losses2 = []
        op2 = optim.SGD(lr=1e-4, params=z2.parameters())
        print ("=============", alpha, "===============")
        for i in range(5000):
            op2.zero_grad()
            yh = z2(a).view(-1)
            loss_use = loss(yh, b)
            loss_use.backward()
            op2.step()
            all_losses2.append(loss_use.item())
            yh = z2(aa).view(-1)
            loss_use = loss(yh, bb)
            all_test_losses2.append(loss_use.item())
            if i % 5000 == 0:
                print (i, all_losses2[-1], all_test_losses2[-1])
            elif i < 500 and i % 100 == 0:
                print (i, all_losses2[-1], all_test_losses2[-1])
            if i % 500000 == 0:
                torch.save([all_losses_alpha_more2, all_test_losses_alpha_more2], f'v8_alpha_experiment_run2_{alpha}_all_alphas_{j}.pt')
        all_losses_alpha_more2[alpha] = all_losses2
        all_test_losses_alpha_more2[alpha] = all_test_losses2
        torch.save([all_losses_alpha_more2, all_test_losses_alpha_more2], f'v8_alpha_experiment_run2_all_alphas_{j}.pt')


In [17]:
check_alphas = sorted([j for j in all_losses_alpha_more2.keys()])
conv_rates = []
alpha = -2
for alpha in check_alphas:
    res = (all_losses_alpha_more2[alpha][1000]/all_losses_alpha_more2[alpha][100])**(1./900)
    conv_rates.append(res)

In [18]:
alpha = 0
const = 1 - (all_losses_alpha_more2[alpha][1000]/all_losses_alpha_more2[alpha][100])**(1./900)

In [19]:
pred_thm_alphas = np.linspace(-10, 0.5, 100)
pred_thm_convs = 1 - const*(1 - pred_thm_alphas)**2 / (1 + pred_thm_alphas**2)

In [20]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.plot(check_alphas, conv_rates, marker='o', linewidth = 4, markersize=8)
ax.plot(pred_thm_alphas, pred_thm_convs, linewidth = 4, markersize=8)
ax.legend(['Experimentally estimated\n convergence rate', 'Theoretical upper bound\n of convergence rate'], fontsize=24)
ax.grid()
ax.set_xlabel("$\\alpha$", fontsize=24)
ax.set_xticklabels(['', '-10', '-8', '-6', '-4', '-2', '0'], fontsize=18)
ax.set_yticklabels(['', '0.9970', '0.9975', '0.9980', '0.9985', '0.9990', '0.9995'], fontsize=18)
# ax.set_ylabel("Convergence Rate", fontsize=24)
fig.savefig('plot_for_all_alphas', dpi=600)